In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import glob


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df_info  = pd.read_csv('SPindex.csv')
df_info = df_info.set_index('Symbol')
df_info.head()

,Security,GICS Sector,GICS Sub Industry,CIK
Symbol,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,66740
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1800
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,1467373


In [3]:
def check_file_existance(file_ids , tmpl):
    files = []
    path = tmpl.format(subject = '*')
    directory_files = glob.glob(path , recursive=True)

    for f_name in file_ids:
        filename = tmpl.format(subject = f_name)
        if filename in directory_files:
            files.append((f_name , filename))
        else:
            files.append((f_name , 'None'))
    return files


## Load Data From Files

In [4]:
def loadData(files , from_date , to_date):
    total  =  []
    
    for i in range(len(files)):
        file_id = files[i][0]
        filename = files[i][1]


        df = pd.read_csv(filename)
        df = df.loc[ (df['Date'] >= from_date) & (df['Date'] <= to_date)]
        
        df.insert(0,'Symbol',file_id)
        total.append(df)
    
    return total

In [5]:
symbol_names = list(df_info.index)
files = check_file_existance(symbol_names , 'stocks\{subject}.csv')
total = loadData(files , '2019-01-01' , '2019-12-31')

## Create File Base Percent of Price Change - Index By Date

In [6]:
def create_change_df_baseDate(total):
    for df in total:
        df.dropna(inplace = True)       
        df['Close_Rolling'] = df['Adj Close'].rolling(5).mean()
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Date', columns='Symbol', values='Close_Rolling')
    
    
    # column name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset.columns[dataset.isna().any()].tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.columns:
            dataset.drop(symbol , axis = 1, inplace = True)       
        
    return dataset

In [7]:
data_perdate = create_change_df_baseDate(total)
data_perdate.to_csv('stocks/total_data_changePercent_Date.csv')
data_perdate.head()

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-08,65.097247,31.480955,159.453345,146.453381,80.841093,72.373422,311.782001,66.574182,137.917685,225.679999,...,85.190263,65.159983,38.413309,19.687399,65.011599,89.023645,101.407343,153.542001,40.388574,83.730052
2019-01-09,65.801427,31.665963,160.749701,145.549625,80.583562,72.736900,317.070001,66.562477,138.951849,227.851996,...,85.579369,65.663883,38.761873,20.024657,65.236047,88.964893,101.431108,154.336002,40.764735,84.076773
2019-01-10,67.112866,32.055662,161.191776,147.825681,80.929959,73.248106,323.638000,67.120416,141.151404,232.265997,...,86.907013,66.296989,39.183715,20.496050,66.157472,89.483862,102.082681,157.030002,41.224911,84.936641
2019-01-11,68.086060,32.008426,161.313251,148.615732,80.792126,73.702942,325.709998,67.358421,142.162061,234.537997,...,87.585498,66.402200,39.520397,20.819894,66.626062,89.382030,102.165860,159.076001,41.486507,85.194202
2019-01-14,68.659694,31.764372,160.831351,149.021536,79.963240,74.113072,325.800000,67.252408,142.822119,235.597998,...,87.741922,66.422505,39.591695,21.090084,67.258064,89.115691,101.926228,160.740002,41.778651,85.041644


## Create File Base Percent of Price Change - Index By Symbol

In [8]:
def create_change_df_baseSymbol(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close_Rolling'] = df['Adj Close'].rolling(5).mean()        
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Symbol', columns='Date', values='Close_Rolling')
    
    
    #symbol name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset[dataset.isna().any(axis=1)].index.tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.index:
            dataset.drop(symbol , axis = 0, inplace = True)
        
        
    return dataset

In [9]:
data_perSymbol = create_change_df_baseSymbol(total)
data_perSymbol.to_csv('stocks/total_data_changePercent_Symbol.csv')
data_perSymbol.head()

Date,2019-01-14,2019-01-15,2019-01-16,2019-01-17,2019-01-18,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-28,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
Symbol,,,,,,,,,,,,,,,,,,,,,
A,68.659694,69.207608,69.361896,69.543875,69.850471,70.161023,70.368719,70.794000,71.353786,71.618842,...,83.677893,83.679884,83.620163,83.914789,84.076038,84.344786,84.746910,84.928065,84.906026,84.951842
AAL,31.764372,31.701391,31.587237,31.890335,32.317430,32.520153,32.431585,32.592976,32.868521,33.380246,...,27.550188,27.777349,27.942738,28.253589,28.528573,28.707910,29.040680,29.018761,28.871306,28.747763
AAP,160.831351,160.883127,160.486853,160.727798,162.358691,163.435999,164.242480,163.021799,160.421127,157.882187,...,155.244272,155.668912,155.752530,156.344638,157.646619,158.175125,158.434393,158.807343,159.006781,159.044675
AAPL,149.021536,149.476358,149.795911,150.199753,151.087827,151.734766,151.901398,151.462256,151.834738,151.732797,...,274.135553,275.920471,277.623804,278.477460,279.301276,280.069360,282.093066,284.039166,286.442938,288.363165
ABBV,79.963240,79.194032,78.974455,78.987061,79.401961,80.117938,80.659509,80.720093,79.497426,77.228326,...,86.091547,86.602910,86.610686,86.892615,87.052051,87.007330,87.104547,87.188153,87.038440,86.705959
